<h1>SVM</h1>

<img src="./img/svm.png">

In [5]:
import numpy as np
from sklearn import preprocessing, cross_validation, svm
import pandas as pd

df = pd.read_csv('dataset/breast_cancer.txt')
df.replace('?',-9999,inplace=True)

# remove useless data, we can also do drop na but that might reduce data set
df.drop(['id'],1,inplace=True)  

x = np.array(df.drop(['class'],1))
y = np.array(df['class'])


x_train, x_test, y_train, y_test = cross_validation.train_test_split(x,y,test_size=0.2)


# train classifier
clf = svm.SVC()
clf.fit(x_train,y_train)

accuracy = clf.score(x_test, y_test)

print(accuracy)

# predict something

example_measures = np.array([[4,2,1,1,1,2,3,4,1],[4,2,1,2,2,2,3,4,1]])
example_measures = example_measures.reshape(len(example_measures),-1)
prediction = clf.predict(example_measures)

print(prediction)


0.9
[2 2]


In [20]:
from IPython.display import YouTubeVideo
YouTubeVideo('https://www.youtube.com/watch?v=VngCRWPrNNc&index=21&list=PLQVvvaa0QuDfKTOs3Keq_kaG2P55YRn5v')
# we need W(normal to hyperplane) and b(the bias) to solve the equation
# U.W + b >= 0 means one class and otherwise other class
# YouTubeVideo('https://www.youtube.com/watch?v=VngCRWPrNNc&index=22&list=PLQVvvaa0QuDfKTOs3Keq_kaG2P55YRn5v')

<h1>Design SVM from Scratch</h1>

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
style.use('ggplot')

class Support_Vector_Machine:
    def __init__(self, visualization=True):
        self.visualization = visualization
        self.colors = {1:'r',-1:'b'}
        if self.visualization:
            self.fig = plt.fugure()
            self.as  = self.fig.add_subplot(1,1,1)
    
    # training 
    def fit(self, data):
        self.data = data
        # {||w|| : [w, b]}
        opt_dict = {}
        # transforms are for w
        tranforms = [[1, 1],
                     [-1, 1],
                     [-1, -1],
                     [1, -1]]
        all_data = []
        for yi in self.data:
            for featureset in self.data[yi]:
                for feature in featureset:
                    all_data.append(feature)
                    
        self.max_feature_value = max(all_data)
        self.min_feature_value = min(all_data)
        all_data = None
        
        step_size = [self.max_feature_value * 0.1,
                     self.max_feature_value * 0.01,
                     # point of expense
                     self.max_feature_value * 0.001]
        # extremely expensize
        b_range_multiple = 5
        # 
        b_multiple = 5
        # first element in W
        latest_optimum = self.max_feature_value * 10
        
        for step in step_sizes:
            w 
        
        
    
    # test
    def predict(self, features):
        # sign(x.w + b)
        classfication = np.sign(np.dot(np.array(features),self.w) + self.b)
        return classification


data_dict = {-1: np.array([[1,7],
                           [2,8],
                           [3,8]]),
              1: np.array([[5,1],
                           [6,-1],
                           [7,3],])}
